In [ ]:
import pymongo

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
client = pymongo.MongoClient(
    
)

In [ ]:
db = client['extractData']

In [ ]:
collection = db['categories.tran']

In [ ]:
collection.count()

In [ ]:
docs = collection.find({'url': {'$exists': True, '$ne': ''}, 'companyInfo': {'$exists': True}},
                       {'url': 1, 'companyInfo.company_id': 1})

In [ ]:
docs.count()

In [ ]:
results = [[doc['_id'], doc['url'], doc['companyInfo'].get('company_id')] for doc in docs]

In [ ]:
df = pd.DataFrame(results, columns=['_id', 'url', 'company_id'])

In [ ]:
df = df[df.url != 'na'].reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
def dedup(value):
    np.unique(value)
#     value.apply()

In [ ]:
def count_distinct(x):
    return len(np.unique())

In [ ]:
agg_ = df.groupby(['url'], as_index=False).agg({'_id': 'count', 'company_id': pd.Series.nunique})

In [ ]:
agg_.head()

In [ ]:
agg_ = agg_[(agg_['_id'] > 1) & (agg_['_id'] != agg_['company_id'])].reset_index(drop=True)

In [ ]:
agg_.shape

In [ ]:
agg_.head()

In [ ]:
agg_ = agg_.rename(columns={'_id': 'nb_docs', 'company_id': 'nb_unique_comps'})

In [ ]:
agg_.head()

In [ ]:
agg_ = agg_.merge(df, on='url', how='left')

In [ ]:
agg_.head()

In [ ]:
_unique = agg_[agg_.company_id.isnull() == True]

In [ ]:
_unique.nb_docs.value_counts()

In [ ]:
_unique.shape

In [ ]:
_unique.url.nunique()

In [ ]:
_unique.head()

In [ ]:
df.isnull().sum()

In [ ]:
db.create_collection('categories.to_delete')

In [ ]:
d

In [ ]:
def insert(row):
    return row.to_dict()

In [ ]:
to_delete = _unique.apply(insert, axis=1)

In [ ]:
db['categories.to_delete'].insert_many(to_delete.tolist())

In [ ]:
db['categories.to_delete'].count()

In [ ]:
def delete(doc):
    db['categories.tran'].delete_one({'_id': doc})

In [ ]:
_unique['_id'].apply(delete)

In [ ]:
139275-9818